In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
from tqdm.notebook import tqdm

Using TensorFlow backend.


In [3]:
print(keras.__version__)
print(tf.__version__)

2.3.1
2.1.0


Define Model

In [4]:
yolo_model = load_model("./model_data/yolo.h5", compile = False)

In [5]:
yolo_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
#class_names = read_classes("./model_data/coco_classes.txt")
#anchors = read_anchors("./model_data/yolo_anchors.txt")
#image_shape = (720., 1280.)   

In [ ]:
#yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [ ]:
transfer_model = keras.Sequential()
transfer_model.add(yolo_model)
transfer_model.add(Conv2D(5, (5, 10), input_shape=(425, 19, 19), padding='valid',))

In [ ]:
transfer_model.summary()

Formulate Training Samples 300 samples

In [ ]:
X_train_orig = np.zeros((300,608,608,3))
file_path = "./images/render/render"
file_ext = ".png"
# frame_range = np.arange(start_frame, end_frame+1)
# range_length = int(len(frame_range))
for i in tqdm(range(300)):
    file_name = file_path+str(i+1).zfill(4)+file_ext
    image, image_data = preprocess_image(file_name, model_image_size = (608, 608))
    X_train_orig[i] = image_data

In [ ]:
X_train_orig.shape

In [ ]:
yolo_coord = np.load('box_coord_dict.npy', allow_pickle = True)
Y_train_orig = np.zeros((300,15,10,5))
for i in tqdm(range(300)):
    Y_train_orig[i] = yolo_coord.item().get(i+1)

In [ ]:
Y_train_orig.shape

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   
sess = tf.compat.v1.Session(config=config)

Start Training

In [ ]:
model = transfer_model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_orig, Y_train_orig, epochs = 15, batch_size = 16)

In [ ]:
from tensorflow.python.platform import build_info as tf_build_info
print(tf_build_info.cuda_version_number)
# 9.0 in v1.10.0
print(tf_build_info.cudnn_version_number)
# 7 in v1.10.0